<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/%E3%82%A2%E3%83%AA%E3%81%A8%E3%83%8F%E3%83%81%E3%81%AE%E3%82%AF%E3%83%A9%E3%82%B9%E5%88%86%E3%81%91%E8%BB%A2%E7%A7%BB%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import urllib.request
import zipfile

# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# ImageNetのclass_indexをダウンロードする
# Kerasで用意されているものです
# https://github.com/fchollet/deep-learning-models/blob/master/imagenet_utils.py

url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = os.path.join(data_dir, "imagenet_class_index.json")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)
    
# 1.3節で使用するアリとハチの画像データをダウンロードし解凍します
# PyTorchのチュートリアルで用意されているものです
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
import glob 
import os.path as osp 
import random 
import numpy as np 
import json 
from PIL import Image 
from tqdm import tqdm 
import matplotlib.pyplot as plt 
%matplotlib inline 

import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.utils.data as data 
import torchvision 
from torchvision import models , transforms 

In [0]:
# 入力画像の前処理をするクラス
# 訓練時と推論時で処理が異なる


class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)

In [72]:
#アリとハチの画像へのファイルパスリストを作成する
def make_datapath_list(phase='train'):
  rootpath = './data/hymenoptera_data/'
  target_path = osp.join(rootpath+phase+'/**/*.jpg')
  print(target_path)
  
  path_list = []
  
  #globを利用してサブディレクトリまでパスを取得する
  for path in glob.glob(target_path):
    path_list.append(path)
  
  return path_list

#実行
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

train_list

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


['./data/hymenoptera_data/train/bees/174142798_e5ad6d76e0.jpg',
 './data/hymenoptera_data/train/bees/3090975720_71f12e6de4.jpg',
 './data/hymenoptera_data/train/bees/198508668_97d818b6c4.jpg',
 './data/hymenoptera_data/train/bees/472288710_2abee16fa0.jpg',
 './data/hymenoptera_data/train/bees/457457145_5f86eb7e9c.jpg',
 './data/hymenoptera_data/train/bees/98391118_bdb1e80cce.jpg',
 './data/hymenoptera_data/train/bees/774440991_63a4aa0cbe.jpg',
 './data/hymenoptera_data/train/bees/476347960_52edd72b06.jpg',
 './data/hymenoptera_data/train/bees/1097045929_1753d1c765.jpg',
 './data/hymenoptera_data/train/bees/3100226504_c0d4f1e3f1.jpg',
 './data/hymenoptera_data/train/bees/513545352_fd3e7c7c5d.jpg',
 './data/hymenoptera_data/train/bees/2861002136_52c7c6f708.jpg',
 './data/hymenoptera_data/train/bees/452462695_40a4e5b559.jpg',
 './data/hymenoptera_data/train/bees/342758693_c56b89b6b6.jpg',
 './data/hymenoptera_data/train/bees/2358061370_9daabbd9ac.jpg',
 './data/hymenoptera_data/train/bees

In [74]:
#Datasetの作成

class HymenopteraDataset(data.Dataset):
  """
  アリとハチの画像のDatasetクラス.PytorchのDatasetクラスを継承
  """
  def __init__(self,file_list,transform=None,phase='train'):
    self.file_list = file_list
    self.transform = transform 
    self.phase = phase
    
  def __len__(self):
    """画像の枚数を返す"""
    return len(self.file_list)
  
  def __getitem__(self,index):
    '''
    前処理をした画像のTensoor形式のデータとラベルを取得
    '''
    img_path = self.file_list[index]
    img = Image.open(img_path)
    
    #画像の前処理を実施
    img_transformed = self.transform(
        img,self.phase
    ) #torch.Size([3,224,224])
    
    #画像のラベルをファイル名から抜き出す
    if self.phase == 'train':
      label = img_path[30:34]
    elif self.phase == 'val':
      label = img_path[28:32]
    
    #ラベルを数値に変換
    if label == 'ants':
      label = 0
    elif label == 'bees':
      label = 1
      
    return img_transformed, label
  
 #実行
train_dataset = HymenopteraDataset(
      file_list = train_list,transform=ImageTransform(size,mean,std),phase='train'
    )
val_dataset = HymenopteraDataset(
      file_list = val_list,transform=ImageTransform(size,mean,std),phase='val'
    )
    
#indexの確認
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])
     

torch.Size([3, 224, 224])
1


In [76]:
#1DataLoader を設定
batch_size = 32 

#DataLoaderを作成
train_loader = torch.utils.data.DataLoader(
train_dataset,batch_size = batch_size,shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
val_dataset,batch_size=batch_size,shuffle=False
)

#辞書型にまとめる
dataloaders_dict = {'train':train_loader,'val':val_dataloader}

#動作確認
batch_iterator = iter(dataloaders_dict['train']) #イテレータに変換
inputs , labels = next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 0])


In [78]:
#学習済みのVGG-16モデルをロードする
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

#最後の出力層を今回の二つに付け替える
net.classifier[6] = nn.Linear(in_features = 4096,out_features=2)

#訓練モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [0]:
#損失関数を定義
criterion = nn.CrossEntropyLoss()

In [82]:
#最適化手法設定する
#転移学習で学習させるパラメータを変数params_to_updateに格納する
params_to_update = []

#学習するパラメータ名
update_param_names=['classifier.6.weight','classifier.6.bias']

#学習させるパラメータ以外は勾配計算をなくし,変化しないように設定
for name,param in net.named_parameters():
  if name in update_param_names:
    param.requires_grad = True
    params_to_update.append(param)
    print(name)
  else:
    param.requires_grad = False

print('params_to_updateの中身を確認します')
print(params_to_update)

classifier.6.weight
classifier.6.bias
params_to_updateの中身を確認します
[Parameter containing:
tensor([[-0.0069,  0.0014,  0.0033,  ...,  0.0050,  0.0072,  0.0087],
        [ 0.0053,  0.0033, -0.0129,  ..., -0.0073,  0.0155, -0.0039]],
       requires_grad=True), Parameter containing:
tensor([-4.8217e-03,  6.2063e-05], requires_grad=True)]


In [0]:
#最適化アルゴリズムの設定
optimizer = optim.SGD(params=params_to_update,lr=0.001,momentum=0.9)

In [0]:
#モデルを学習する関数を定義する
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
  
  #epochのループ
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    
    print('--------------------')
    
    #epochごとの学習と検証のループ
    for phase in ['train','val']:
      if phase == 'train':
        net.train()  #モデルを訓練モードに
      else : 
        net.eval()   #モデルを検証モードに
        
      epoch_loss = 0.0 #epochの損失和
      epoch_corrects = 0 #epochの正解数
      
      #未学習の検証性能を確かめるためにepoch=0の訓練は省略
      if (epoch == 0) and (phase == 'train'):
        continue
      #データローダーからミニバッチを取り出すループ
      for inputs,labels in tqdm(dataloaders_dict[phase]):
        
        #optimizerを初期化
        optimizer.zero_grad()
        
        #順伝搬(forward)計算
        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs)
          loss = criterion(outputs,labels) #損失を計算
          _,preds = torch.max(outputs,1) #ラベルを予測
          
          #訓練時はバックプロバケーション
          if phase == 'train':
            loss.backward()
            optimizer.step()
            
          #イテレーション結果の計算
          #lossの合計を計算
          epoch_loss += loss.item() * inputs.size(0)
          #正解数の合計を更新
          epoch_corrects += torch.sum(preds == labels.data)
       
      #epochごとのlossと正解率を表示
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
      
      print('{} Los:{:.4f} Acc{:.4f}'.format(phase,epoch_loss,epoch_acc))

In [87]:
#学習,検証を実行
num_epochs = 2
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs=num_epochs)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/2
--------------------


  0%|          | 0/8 [00:00<?, ?it/s]

val Los:0.9159 Acc0.2353
Epoch 2/2
--------------------


  0%|          | 0/5 [00:00<?, ?it/s]

train Los:0.5552 Acc0.6626


100%|██████████| 5/5 [01:25<00:00, 16.64s/it]

val Los:0.1806 Acc0.9412
